# Mathematical Framework of Loihi
The purpose of this notebook is display the mathematical framework of the Loihi Chip and to simplify the parameter tuning of future compartments and networks

## Neural Model 
Loihi uses the leaky-integrate-and-fire model which has two internal state variables: 
### 1) $u_i(t)$: the synaptic response current
$$
u_i(t) = u_i(t - 1) \cdot (2^{12} - \delta_i^{(u)}) \cdot 2^{-12} + 2^{6+wgtExp} \sum_jw_{ij}\cdot s_j(t)
$$
- $i$ indicates the index of the post-syntapic neuron (in Loihi neuron are represented as compartments)
- $j$ indicates the index of the pre-syntapic neuron
- $\delta_i^{(u)}$ represents the current decay (Default = 4096)
$$
compartmentCurrentDecay = (1/\tau)*2^{12} 
$$
$$
\tau = e^{(-t/\tau)}
$$
- $u_i(t)$ is the compartment's state current at timestep $t$. The compartment current integrates incoming weighted spikes from the dendritic accumulators and possibly inputs from other compartments but decays exponentially otherwise
### 2) $v_t(t)$: the membrane voltage potential
$$
v_i(t) = v_i(t-1) \cdot (2^{12} - VoltageDecay) * 2^{-12} + u_i(t) + (biasMant*2^{biasExp})
$$
- Voltage decay is defined by (default = 0):
$$
VoltageDecay = (1/\tau) \cdot 2^{12}
$$
- the compartment voltage $(v_i(t))$ integrates the compartment current $(u_i(t))$, the compartment bias, and possibly inputs from other compartments
- the firing rate comes from setting the voltage threshold (default 6400): 
$$
vTh = vthMant * 2^6
$$




In [24]:
def get_vth(vthMant):
    return vthMant * (2 **6)

def get_vthMant(vTh):
    return vTh /(2 ** 6)

def get_curr_decay_tau(currentDecay):
    return (2 ** 12) / currentDecay

def get_curr_decay(tau):
    return (1 / tau) * (2 ** 12)

def act_voltage_decay(voltageDecay):
    return ((2**12) - voltageDecay) / (2**12)

def get_voltage_decay(tau):
    return (1 / tau) * (2 ** 12)





## Synaptic Connection Model
- Weight $(w)$ can take on a range of values of [-256, 256]
- The below formulas break down how synatpic weights are accumulated into the compartment current
$$
numLsbBits = 8 - (numWeightbits - IS_MIXED) \\
actWeight = (weight >> numLsbBits) << numLsbBits
$$
- `numWeightBits` specifies the number of bits and therefore the precision of a synaptic `weight`. It can take values of `0,1,2,3,4,5,6,7,8`
- Before weight is accumulated to a current value, an additional exponential scalling is performed: 
$$
2^{6 + wgtExp}
$$
- In summary the weight component that gets integrated to current is as follows (assuming 8 bit resolution)
$$
w_i = w * 2 ^{6 + wgtExp}
$$

In [82]:
def calculate_actual_weight(weight, num_weight_bits, wgt_exp, is_mixed):
    """
    Calculate the actual weight value that gets integrated into the compartment current.
    
    Parameters:
    weight (int): The synaptic weight in the range of [-256, 256].
    num_weight_bits (int): The number of bits specifying the precision of the synaptic weight. Can take values from 0 to 8.
    wgt_exp (int): The weight exponent used for additional exponential scaling.
    is_mixed (int): Indicator if the mixed precision mode is used.
    
    Returns:
    float: The actual weight value that gets integrated into the compartment current.
    """
    # Calculate the number of least significant bits
    num_lsb_bits = 8 - (num_weight_bits - is_mixed)
    
    # Calculate the actual weight by shifting
    act_weight = (weight >> num_lsb_bits) << num_lsb_bits
    
    # Calculate the weight component that gets integrated into the current
    weight_component = act_weight * (2 ** (6 + wgt_exp))
    
    return weight_component

# Example usage
weight = 200
num_weight_bits = 8
wgt_exp = 0
is_mixed = 0

actual_weight = calculate_actual_weight(weight, num_weight_bits, wgt_exp, is_mixed)
print("Actual Weight:", actual_weight)


Actual Weight: 12800


# Poisson Spike Trains 

In [9]:
import numpy as np
import ipywidgets as widgets

# Define functions for calculations
def get_vth(vthMant):
    return vthMant * (2 ** 6)

def update_vth(change):
    vth = get_vth(change['new'])
    vth_output.value = f"Vth = {vth} mV"

def get_vthMant(vTh):
    return vTh / (2 ** 6)

def get_curr_decay_tau(currentDecay):
    return (2 ** 12) / currentDecay

def get_curr_decay(tau):
    return (1 / tau) * (2 ** 12)

def act_voltage_decay(voltageDecay):
    return ((2**12) - voltageDecay) / (2**12)

def get_voltage_decay(tau):
    return (1 / tau) * (2 ** 12)

def calculate_actual_weight(weight, num_weight_bits, wgt_exp, is_mixed):
    num_lsb_bits = 8 - (num_weight_bits - is_mixed)
    act_weight = (weight >> num_lsb_bits) << num_lsb_bits
    weight_component = act_weight * (2 ** (6 + wgt_exp))
    return weight_component

def update_weight(change):
    weight_value = change['new']
    act_weight = calculate_actual_weight(weight_value, num_weight_bits=8, wgt_exp=1, is_mixed=0)
    weight_output.value = f"Weight = {act_weight}"

# Create a slider for VthMant input
vthMant_input = widgets.IntSlider(value=1, min=1, max=100, step=1, description='VthMant:')

# Create a text widget to display the output
vth_output = widgets.Text(value=f"Vth = {get_vth(vthMant_input.value)} mV", description='Voltage:', disabled=True)

# Link the slider to the update function using observe
vthMant_input.observe(update_vth, names='value')

# Create a slider for weight input
weight_input = widgets.IntSlider(value=0, min=-256, max=256, step=1, description='Weight:')

# Create a text widget to display the weight output
weight_output = widgets.Text(value=f"Weight = {calculate_actual_weight(weight_input.value, num_weight_bits=8, wgt_exp=1, is_mixed=0)}", 
                             description='Weight Component:', disabled=True)

# Link the slider to the update function for weight
weight_input.observe(update_weight, names='value')

header = widgets.HTML(
    value= "<b> Calculate Loihi Values and Tune Parameters <\b>"
)

calc_button = widgets.Button(
    value=False,
    description='Calculate Values',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check'
)

out = widgets.Output(layout={'border': '1px solid black'})

with out: 
    display(header)
    # Display the slider and output for Voltage Threshold
    display(vthMant_input, vth_output)

    # Display the slider and output for Weight
    display(weight_input, weight_output)

    display(calc_button)

# TODO: callback function
#calc_button.on_click(some function)




In [12]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display

# Define functions for calculations
def get_vth(vthMant):
    return vthMant * (2 ** 6)

def update_vth(change):
    vth = get_vth(change['new'])
    vth_output.value = f"Vth = {vth} mV"

def get_vthMant(vTh):
    return vTh / (2 ** 6)

def get_curr_decay_tau(currentDecay):
    return (2 ** 12) / currentDecay

def get_curr_decay(tau):
    return (1 / tau) * (2 ** 12)

def act_voltage_decay(voltageDecay):
    return ((2**12) - voltageDecay) / (2**12)

def get_voltage_decay(tau):
    return (1 / tau) * (2 ** 12)

def calculate_actual_weight(weight, num_weight_bits, wgt_exp, is_mixed):
    num_lsb_bits = 8 - (num_weight_bits - is_mixed)
    act_weight = (weight >> num_lsb_bits) << num_lsb_bits
    weight_component = act_weight * (2 ** (6 + wgt_exp))
    return weight_component

def update_weight(change):
    weight_value = change['new']
    act_weight = calculate_actual_weight(weight_value, num_weight_bits=8, wgt_exp=1, is_mixed=0)
    weight_output.value = f"Weight = {act_weight}"

# Create a slider for VthMant input
vthMant_input = widgets.IntSlider(value=1, min=1, max=100, step=1, description='VthMant:')
# Create a text widget to display the output
vth_output = widgets.Text(value=f"Vth = {get_vth(vthMant_input.value)} mV", description='Voltage:', disabled=True)
# Link the slider to the update function using observe
vthMant_input.observe(update_vth, names='value')
# Create a slider for weight input
weight_input = widgets.IntSlider(value=0, min=-256, max=256, step=1, description='Weight:')
# Create a text widget to display the weight output
weight_output = widgets.Text(value=f"Weight = {calculate_actual_weight(weight_input.value, num_weight_bits=8, wgt_exp=1, is_mixed=0)}", 
                             description='Weight Component:', disabled=True)
# Link the slider to the update function for weight
weight_input.observe(update_weight, names='value')


header = widgets.HTML(
    value= "<b> Calculate Loihi Values and Tune Parameters <\b>"
)

calc_button = widgets.Button(
    description='Calculate Values',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger'
    tooltip='Description',
    icon='check'
)

out = widgets.Output(layout={'border': '1px solid black'})

# Properly use the Output widget by displaying widgets outside its context
with out:
    display(header)
    # Display the slider and output for Voltage Threshold
    display(vthMant_input, vth_output)
    # Display the slider and output for Weight
    display(weight_input, weight_output)

    display(calc_button)

# Display the output widget
display(out)

# Example callback for button
def on_calc_button_clicked(b):
    with out:
        print("Calculation done!")  # Just as an example action

calc_button.on_click(on_calc_button_clicked)

Output(layout=Layout(border='1px solid black'))